# Step 0.0. Install LightAutoML

In [ ]:
pip install -U lightautoml

# Step 0.1. Import necessary libraries 

In [ ]:
# Standard python libraries
import os
import time
import re

# Installed libraries
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
import torch

# Imports from our package
from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
from lightautoml.dataset.roles import DatetimeRole
from lightautoml.tasks import Task
# from lightautoml.utils.profiler import Profiler

# Step 0.2. Parameters 

In [ ]:
N_THREADS = 4 # threads cnt for lgbm and linear models
N_FOLDS = 5 # folds cnt for AutoML
RANDOM_STATE = 42 # fixed random state for various reasons
TEST_SIZE = 0.2 # Test size for metric check
TIMEOUT = 600 # Time in seconds for automl run

# Step 0.3. Fix torch number of threads and numpy seed 

In [ ]:
np.random.seed(RANDOM_STATE)
torch.set_num_threads(N_THREADS)

# Step 0.4. Data load 

In [ ]:
%%time

train_data = pd.read_csv('../input/santander-customer-satisfaction/train.csv')
train_data.head()

In [ ]:
test_data = pd.read_csv('../input/santander-customer-satisfaction/test.csv')
test_data.head()

In [ ]:
submission = pd.read_csv('../input/santander-customer-satisfaction/sample_submission.csv')
submission.head()

In [ ]:
target_name = 'TARGET'
submission_target_name = 'TARGET'


# Step 0.5. Add new features

# Step 0.6. Data splitting for train-test 

In [ ]:
tr_data, te_data = train_test_split(train_data, 
                                     test_size=TEST_SIZE, 
                                     stratify=train_data[target_name], 
                                     random_state=RANDOM_STATE)
print('Data splitted. Parts sizes: tr_data = {}, te_data = {}'.format(tr_data.shape, te_data.shape))

In [ ]:
train_data[target_name].value_counts(), tr_data[target_name].value_counts(), te_data[target_name].value_counts(), 

# ========= AutoML preset usage =========


## Step 1. Create Task

In [ ]:
%%time
def acc_score(y_true, y_pred, **kwargs):
    return accuracy_score(y_true, (y_pred > 0.5).astype(int), **kwargs)

def f1_metric(y_true, y_pred, **kwargs):
    return f1_score(y_true, (y_pred > 0.5).astype(int), **kwargs)

def f1_loss(y_true, y_pred, **kwargs):
    return -f1_score(y_true, (y_pred > 0.5).astype(int), **kwargs)

def roc_auc_metric(y_true, y_pred, **kwargs):
    return roc_auc_score(y_true, (y_pred > 0.5).astype(int), **kwargs)

def roc_auc_loss(y_true, y_pred, **kwargs):
    return -roc_auc_score(y_true, (y_pred > 0.5).astype(int), **kwargs)


# task = Task('binary')
# task = Task('binary', metric = f1_metric)
# task = Task('binary', metric = 'roc_auc')
# task = Task('binary', loss = 'f1_metric', metric = 'f1_metric')
task = Task('binary', 
#             loss = {'f1_loss': f1_loss}, 
#             metric = 'auc',
#             metric = 'accuracy',
            metric = 'logloss',
           )


## Step 2. Setup columns roles

In [ ]:
%%time

roles = {
    'target': target_name,
    'drop': ['ID'],
}

## Step 3. Create AutoML from preset and train on 80% of data

In [ ]:
# %%time 

# automl = TabularAutoML(task = task, 
#                        timeout = TIMEOUT,
#                        cpu_limit = N_THREADS,
#                        general_params = {'use_algos': [['linear_l2', 'lgb', 'lgb_tuned']]},
#                        reader_params = {'n_jobs': N_THREADS})
# oof_pred = automl.fit_predict(tr_data, roles = roles)
# print('oof_pred:\n{}\nShape = {}'.format(oof_pred[:10], oof_pred.shape))

## Step 4. Predict to validation data and check scores

In [ ]:
# %%time

# test_pred = automl.predict(te_data)
# print('Prediction for test data:\n{}\nShape = {}'.format(test_pred[:10], test_pred.shape))

# print('Check scores...')
# print('OOF score: {}'.format(acc_score(tr_data[target_name].values, oof_pred.data[:, 0])))
# print('TEST score: {}'.format(acc_score(te_data[target_name].values, test_pred.data[:, 0])))

    Score for `TabularAutoML` is 83.8% accuracy for OOF preds and 82.1% accuracy for validation preds in 2 minutes time. 

## Step 5. Create AutoML with time utilization 

Below we are going to create specific AutoML preset for TIMEOUT utilization (try to spend it as much as possible):

In [ ]:
# %%time 

# automl = TabularUtilizedAutoML(task = task, 
#                        timeout = TIMEOUT,
#                        cpu_limit = N_THREADS,
#                        general_params = {'use_algos': [['linear_l2', 'lgb', 'lgb_tuned']]},
#                        reader_params = {'n_jobs': N_THREADS})
# oof_pred = automl.fit_predict(tr_data, roles = roles)
# print('oof_pred:\n{}\nShape = {}'.format(oof_pred[:10], oof_pred.shape))

## Step 6. Predict to validation data and check scores for utilized automl

In [ ]:
# %%time

# test_pred = automl.predict(te_data)
# print('Prediction for test data:\n{}\nShape = {}'.format(test_pred[:10], test_pred.shape))

# print('Check scores...')
# print('OOF score: {}'.format(acc_score(tr_data[target_name].values, oof_pred.data[:, 0])))
# print('TEST score: {}'.format(acc_score(te_data[target_name].values, test_pred.data[:, 0])))

Score for `TabularUtilizedAutoML` is 85.5% accuracy for OOF preds and 82.7% accuracy for validation preds in 10 minutes. As the validation score is better for `TabularUtilizedAutoML` for this case, we choose it for final model retrain on full train dataset.

## Step 7. Train on full data 

In [ ]:
%%time 

automl = TabularUtilizedAutoML(task = task, 
                       timeout = TIMEOUT,
                       cpu_limit = N_THREADS,
                       general_params = {'use_algos': [['linear_l2', 'lgb', 'lgb_tuned']]},
                       reader_params = {'n_jobs': N_THREADS})
oof_pred = automl.fit_predict(train_data, roles = roles)
print('\n\n\n')
print('oof_pred:\n{}\nShape = {}'.format(oof_pred[:10], oof_pred.shape))

## Step 8. Predict for test data and check OOF score

In [ ]:
%%time

test_pred = automl.predict(test_data)
print('Prediction for test data:\n{}\nShape = {}'.format(test_pred[:10], test_pred.shape))

print('Check scores...')
# print('OOF score: {}'.format(acc_score(train_data[target_name].values, oof_pred.data[:, 0])))
print('OOF score(roc_auc_metric): {}'.format(roc_auc_metric(train_data[target_name].values, oof_pred.data[:, 0])))
print('OOF score(f1_metric): {}'.format(f1_metric(train_data[target_name].values, oof_pred.data[:, 0])))


## Step 9. Prepare submission

In [ ]:
submission[submission_target_name] = (test_pred.data[:, 0] > 0.5).astype(int)
submission.to_csv('automl_2_utilized_600_metric_roc_auc.csv', index = False)

In [ ]:
submission

Submission above scores 79.665% accuracy on public LB.